In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
import time
from datetime import datetime
from multiprocessing import Pool, cpu_count
from scipy.spatial import KDTree
from scipy import interpolate
#  ========================================================================================================
def project_and_get_distances(points, reference_point):
    from pyproj import Transformer
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32648", always_xy=True)
    
    # Project all points
    x, y = transformer.transform(points[:, 0], points[:, 1])
    x0, y0 = transformer.transform(reference_point[0], reference_point[1])
    xy = np.column_stack((x, y))
    ref_xy = np.array([x0, y0])
    distances = np.linalg.norm(xy - ref_xy, axis=1)
    return distances
def dist2ref(df,ref_point,keylong='Longitude',keylat='Latitude'):
    from pyproj import Geod
    '''
    calculate the distance from the spicies coordinate to reference point
    Input is the data frame, the distance is a new column
    '''
    # Define geodetic system (WGS84)
    geod = Geod(ellps='WGS84')
    # Compute forward and back azimuths and distance
    # Function to compute distance
    def compute_distance(row):
        az12, az21, dist = geod.inv(ref_point[0], ref_point[1],row[keylong], row[keylat])
        az_rad = np.deg2rad(az12)
        dx = dist * np.sin(az_rad)  # East-west
        dy = dist * np.cos(az_rad)  # North-south
        return pd.Series([dist, dx, dy], index=['distance','d2x', 'd2y'])  # in meters
    # Apply row-wise
    df[['distance','d2x', 'd2y']] = df.apply(compute_distance, axis=1)
    return df  


def ripley_k_border_corrected(points, reference_point, max_radius, step=1.0, window=None, inner_window=None):
    """
    Ripley's K with border (reduced sample) correction, using predefined inner usable area.

    Args:
        points (ndarray): Nx2 array of point coordinates.
        max_radius (float): Maximum radius to evaluate K.
        step (float): Distance step size.
        window (tuple): (xmin, xmax, ymin, ymax), the full sampling area.
        inner_window (tuple): (xmin, xmax, ymin, ymax), the inner area where circles of radius r fit entirely.

    Returns:
        radii (ndarray): Radii at which K(r) is evaluated.
        K_values (ndarray): Border-corrected K values.
    """
    n = len(points)
    if n < 2:
        raise ValueError("Need at least two points.")

    # Determine full window (outer box)
    if window is None:
        xmin, ymin = np.min(points, axis=0)
        xmax, ymax = np.max(points, axis=0)
    else:
        xmin, xmax, ymin, ymax = window

    # Determine inner usable area
    if inner_window is None:
        # Use dynamic check (original way)
        use_dynamic = True
    else:
        ixmin, ixmax, iymin, iymax = inner_window
        use_dynamic = False

    area = (xmax - xmin) * (ymax - ymin)
    density = n / area
    
    tree = KDTree(points)
    radii = np.arange(step, max_radius + step, step)
    K_values = np.zeros_like(radii)

    for i, r in enumerate(radii):
        if use_dynamic: # border correction
            # Dynamically choose valid points
            valid_points = [p for p in points if (
                p[0] - r >= xmin and p[0] + r <= xmax and
                p[1] - r >= ymin and p[1] + r <= ymax)]
        else: # no border correction
            # Use inner window provided
            valid_points = [p for p in points if (
                p[0] >= ixmin and p[0] <= ixmax and
                p[1] >= iymin and p[1] <= iymax)]

        valid_points = np.array(valid_points)
        if len(valid_points) < 2:
            continue

        count = tree.query_ball_point(reference_point, r, return_length=True)
        # total_neighbors = np.sum(counts) - len(valid_points)
        K_values[i] = (count - 1) / density

    return radii, K_values
    
    
def ripley_k(points, reference_point, radius, area):
    """
    Compute the Ripley-K function for a given reference point.
    + Input: 
    """
    # distances = np.linalg.norm(points - reference_point, axis=1)
    distances = project_and_get_distances(points, reference_point)
    count_within_radius = np.sum(distances <= radius) - 1  # exclude self
    K = (area / len(points)) * count_within_radius
    return K
def ripley_k_full(points, radius, area):
    from scipy.spatial.distance import pdist, squareform
    '''
    This calculate the ripley-K for full data set, not for single point
    '''
    n = len(points)
    dist_matrix = squareform(pdist(points))
    count = np.sum((dist_matrix <= radius)) - n  # subtract self-counts
    K = (area / (n * (n - 1))) * count
    return K
    
def derivative_ripley_k(points, reference_point, radius, area, delta_r=1e-5):
    """
    Compute the derivative of the Ripley-K function at a reference point with respect to the radius.
    """
    K_r = ripley_k(points, reference_point, radius, area)
    K_r_plus_delta = ripley_k(points, reference_point, radius + delta_r, area)
    dK_dr = ((K_r_plus_delta - K_r) / delta_r)
    # print("> Radius:",radius)
    if radius == 0:
        gr = 0
        Lr = 0
    else:
        gr = dK_dr/(2*np.pi*radius)
        Lr = np.sqrt(K_r/np.pi) - radius
    return gr, Lr

# Helper function to apply to each point
def compute_gr_for_point(args):
    '''
    Parallel computing for this GR
    '''
    point, radius, area, step, pointxy = args
    # print(point)
    refpoint=[point[1][2],point[1][3]]
    #
    # radii, gr, Lr = derivative_ripley_k_range(points_global_only_coords, point, radius, area, step, 0.1)
    radii, Kr = ripley_k_border_corrected(pointxy, refpoint, radius, step=1.0, window=None, inner_window=None)
    # ---------------------------------------------------------
    # now calculate derivative dK_dr
    dK_dr = np.gradient(Kr,radii)
    gr = dK_dr/(2 * np.pi * radii); gr[gr < 0] = 0;
    Lr = np.sqrt(Kr / np.pi) - radii;
    # ---------------------------------------------------------
    return pd.DataFrame({
        'Point': np.full(len(radii), point[0]),
        'Longitude': np.full(len(radii), point[1][0]),
        'Latitude': np.full(len(radii), point[1][1]),
        'radius_m': radii,
        'gr': gr,
        'Lr': Lr
    })
    # ---------------------------------------------------------
#  ========================================================================================================
if __name__ == "__main__":
    # Start timing
    t0 = time.time()
    start_time_str = datetime.now().strftime("Start time: %a %b %d %H:%M:%S %Y")
    print(start_time_str)
    # ----------------------------------------------------------------------------
    pwd = os.getcwd()
    #
    infile = os.path.join(pwd,"output","01_all_IVI_sorted.txt")
    #
    all_data_file = os.path.join(pwd,"output","01_all_distances.txt")
    # read the data and kip 2 rows due to format error
    data_ini = pd.read_csv(infile, sep=',',skiprows=0,header=0,)
    all_data = pd.read_csv(all_data_file, sep=',',skiprows=0,header=0,)
    # data_ini.head(20)
    # Select the data that less than 75% of the cumulative IVI
    data_selected = data_ini[data_ini['IVIcu'] <= 75.]; data_selected.reset_index(drop=True,inplace=True)
    # Now filter the data with selected spices 
    all_data_selected = all_data.head(0)
    for i,sourcecodenow in enumerate(data_selected['sourcecode']):
        all_data_now = all_data[all_data['source_code']==sourcecodenow];
        all_data_now.reset_index(drop=True,inplace=True)
        
        all_data_selected = pd.concat([all_data_selected,all_data_now],axis=0)
    all_data_selected.reset_index(drop=True,inplace=True)
    points = all_data_selected[['source_long', 'source_lat']].drop_duplicates().to_numpy()
    # # Area of study area
    # area = area_deg_plain(data['Sourcelong'].min(),data['Sourcelat'].min(),data['Sourcelong'].max(),data['Sourcelat'].max())
    # # radius
    # radius=avg_dist/111320; # in deg
    # #
    dk_dr_values=[];
    radiis=[]
    ref_points_long=[];
    ref_points_lat=[];
    # calculate the average dist
    all_data_selected['pair'] = all_data_selected.apply(lambda row: tuple(sorted([row['source_long'], row['source_lat']])),axis=1)
    all_data_selected_unique = all_data_selected.drop_duplicates(subset='pair')
    avg_dist = all_data_selected_unique['Dist'].mean()
    # radius
    # radius=avg_dist/111320; # in deg
    radius = int(avg_dist);
    area = data_ini['Area'].unique()[0]
    # area = data_ini['Area'].unique()[0]/10000 # convert to hecta
    # Now calculate the Gr for all the data points
    # points_global = all_data_selected[['source_code','source_long', 'source_lat']].drop_duplicates().to_numpy()
    unique_points = all_data_selected[['source', 'source_long', 'source_lat','source2x','source2y']].drop_duplicates()
    # Đổi thành numpy array: mỗi phần tử là (name, (lon, lat))
    points_global = [(row['source'], (row['source_long'], row['source_lat'],row['source2x'],row['source2y'])) for _, row in unique_points.iterrows()]
    #
    points_xy = all_data_selected[['source2x', 'source2y']].drop_duplicates().to_numpy()
    #
    num_cpus = min(40, cpu_count()) # 40 Cpu or max
    print("Calculating GR with critical radius = {} (m), area = {} (m), number of point = {}, using {} CPUs...".format(radius,area,len(points_global),num_cpus))
    # Prepare argument list for parallel pool
    # args_list = [(tuple(point), radius, area, int(avg_dist)+1) for point in points_global]
    args_list = [(point, radius, area, int(avg_dist)+1,points_xy) for point in points_global]

    # Parallel computation
    with Pool(processes=num_cpus) as pool:
        results = pool.map(compute_gr_for_point, args_list)
    # for args_list_now in args_list:
    #     # print(args_list_now)
        
    #     results = compute_gr_for_point(args_list_now) 
    # results
    dataout = pd.concat(results, ignore_index=True)
    
    ref_point=[108.5754,14.4991]
    dataout = dist2ref(dataout, ref_point)
    # # ====================================================================================================================
    dataout.to_csv(os.path.join(pwd,"output","02_species_GR.txt"),index=False,sep='\t',encoding='utf-8')
    print(' >>>> Export the output file: {}'.format(os.path.join(pwd,"output","02_species_GR.txt")))
    dataout['radius_m'] = dataout['radius_m'].apply(lambda x: float(f"{x:.1f}"))
    # End timing
    t1 = time.time()
    end_time_str = datetime.now().strftime("End time: %a %b %d %H:%M:%S %Y")  
    total_time = t1 - t0
    print(end_time_str)
    print("Total process time: {:.2f} seconds".format(total_time))
    print("done!")

Start time: Wed Jul 30 21:19:11 2025
Calculating GR with critical radius = 56 (m), area = 13386.634249231713 (m), number of point = 558, using 40 CPUs...
 >>>> Export the output file: /data/longhv/Projects/VietNga_species_distribution_example/spatial_mapping/output/02_species_GR.txt
End time: Wed Jul 30 21:19:23 2025
Total process time: 11.37 seconds
done!


In [3]:
all_data_selected

,source,source_code,source_long,source_lat,source2x,source2y,source_C,source_DBH,source_S,source_spiccies_code,...,site_long,site_lat,site2x,site2y,site_C,site_DBH,site_S,site_spiccies_code,Dist,pair
0,S0113,A20,108.576470,14.499707,115.342219,67.161125,34.0,10.82,0.009198,A20,...,108.576385,14.499769,106.179489,74.021007,62.5,19.89,0.031081,A1,11.446116,"(14.499707, 108.57647)"
1,S0113,A20,108.576470,14.499707,115.342219,67.161125,34.0,10.82,0.009198,A20,...,108.576357,14.499799,103.161175,77.340312,84.8,26.99,0.057217,A1,15.874308,"(14.499707, 108.57647)"
2,S0113,A20,108.576470,14.499707,115.342219,67.161125,34.0,10.82,0.009198,A20,...,108.576284,14.499337,95.292234,26.222791,53.5,17.03,0.022774,A1,45.584527,"(14.499707, 108.57647)"
3,S0113,A20,108.576470,14.499707,115.342219,67.161125,34.0,10.82,0.009198,A20,...,108.576277,14.499328,94.537661,25.226993,88.5,28.17,0.062319,A1,46.811335,"(14.499707, 108.57647)"
4,S0113,A20,108.576470,14.499707,115.342219,67.161125,34.0,10.82,0.009198,A20,...,108.576259,14.499651,92.597187,60.964970,69.0,21.96,0.037882,A1,23.573900,"(14.499707, 108.57647)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440815,S0323,A27,108.575577,14.499978,19.036833,97.200687,80.5,25.62,0.051562,A27,...,108.576164,14.499496,82.356578,43.815127,38.7,12.32,0.011917,A83,82.821544,"(14.4999785, 108.5755766)"
440816,S0323,A27,108.575577,14.499978,19.036833,97.200687,80.5,25.62,0.051562,A27,...,108.575700,14.500011,32.349675,100.796628,42.0,13.37,0.014036,A83,13.789945,"(14.4999785, 108.5755766)"
440817,S0323,A27,108.575577,14.499978,19.036833,97.200687,80.5,25.62,0.051562,A27,...,108.575632,14.499988,25.030308,98.240745,61.4,19.54,0.029996,A83,6.083047,"(14.4999785, 108.5755766)"
440818,S0323,A27,108.575577,14.499978,19.036833,97.200687,80.5,25.62,0.051562,A27,...,108.576417,14.499298,109.629207,21.907738,45.0,14.32,0.016112,A85,117.796462,"(14.4999785, 108.5755766)"


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
# ======================================================================
'''
Plot the figure for each radius distances
'''
# ======================================================================
pwd = os.getcwd();
#
indata = pd.read_csv(os.path.join(pwd,"output","02_species_GR.txt"),sep="\s+",header=0)
# Output figure
out_fig_dir = os.path.join(pwd,"figures","GR_radius")
os.makedirs(out_fig_dir,exist_ok=True)
# ======================================================================
for i,radius in enumerate(indata['radius_m']):
    print(radius)
    if i > 50:
        break
    # Extract the data
    data_now = indata[indata['radius_m']==radius]
    data_now.reset_index(drop=True,inplace=True)
    # Now plot the figure
    fig = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,np.ceil(int(indata['d2x'].max())/10)*10,0,np.ceil(int(indata['d2y'].max())/10)*10]
    # grid = pygmt.datasets.load_earth_relief(resolution="01s", region=area)
    # fig.grdimage(grid=grid, projection="X5i/5i", frame="a", cmap="geo")
    # fig.colorbar(frame=["a100", "x+lElevation", "y+lm"])
    fig.basemap(region=area, projection="X5i/5i", frame=['xafg+l"Easting(m)"','yafg+l"Northing(m)"', 'WSne+t"G(r) map with radius of {} (m)"'.format(radius)])
    gr_grd = pygmt.surface(x=data_now['d2x'], y=data_now['d2y'], z=data_now['gr'],
                            region=area, spacing='0.5/0.5') # 1 metter per 1 metter
    # pygmt.makecpt(cmap=["0   blue","1   white","{}   red".format(np.ceil((data_now['gr'].max())*10)/10)],series=[0, (np.ceil((data_now['gr'].max())*10)/10), 0.01])
    with open("custom.cpt", "w") as f:
        f.write(
            """
            0	blue	1	white
            1	white	{}	red
            B	blue
            F	white
            N	gray
            """.format(int(data_now['gr'].max()))
            )
    # -------------------------------------------------------------------
    # pygmt.makecpt(cmap=cmap,series=[0, 7, 0.01])
    fig.grdimage(grid=gr_grd,
                 # shading=True,
                 # projection="X5i/5i",
                 cmap="custom.cpt")
    fig.plot(x=data_now['d2x'],y=data_now['d2y'],
             style='c0.1c',
             pen='1p,black',
    )
    fig.colorbar(
        cmap="custom.cpt",
        position="JMR+o0.25i/0.0c+w5.0i/0.25c+n",
        box=None,
        frame=["xaf+lG(r)"],
        # scale=1,  # Adjust the scale
        # B="x10"
    )
    fig.savefig(os.path.join(out_fig_dir,"00_distribution_gr_{}_km.png".format(radius)),crop=True, dpi=300, transparent=False)
    # fig.show()
    # ---------------------------------------------------------------------------------------------------------------------------
    fig1 = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,np.ceil(int(indata['d2x'].max())/10)*10,0,np.ceil(int(indata['d2y'].max())/10)*10]
    # grid = pygmt.datasets.load_earth_relief(resolution="01s", region=area)
    # fig.grdimage(grid=grid, projection="X5i/5i", frame="a", cmap="geo")
    # fig.colorbar(frame=["a100", "x+lElevation", "y+lm"])
    fig1.basemap(region=area, projection="X5i/5i", frame=['xafg+l"Easting(m)"','yafg+l"Northing(m)"', 'WSne+t"L(r) map with radius of {} (m)"'.format(radius)])
    gr_grd = pygmt.surface(x=data_now['d2x'], y=data_now['d2y'], z=data_now['Lr'],
                            region=area, spacing='0.5/0.5') # 1 metter per 1 metter
    # pygmt.makecpt(cmap=["0   blue","1   white","{}   red".format(np.ceil((data_now['gr'].max())*10)/10)],series=[0, (np.ceil((data_now['gr'].max())*10)/10), 0.01])
    with open("custom_Kr.cpt", "w") as f:
        f.write(
            """
            {}	blue	0	white
            0	white	{}	red
            B	blue
            F	white
            N	gray
            """.format((data_now['Lr'].min()),(data_now['Lr'].max()))
            )
    # -------------------------------------------------------------------
    # pygmt.makecpt(cmap=cmap,series=[0, 7, 0.01])
    fig1.grdimage(grid=gr_grd,
                 # shading=True,
                 # projection="X5i/5i",
                 cmap="custom_Kr.cpt")
    
    fig1.plot(x=data_now['d2x'],y=data_now['d2y'],
             style='c0.1c',
             pen='1p,black',
    )
    
    fig1.colorbar(
        cmap="custom_Kr.cpt",
        position="JMR+o0.25i/0.0c+w5.0i/0.25c+n",
        box=None,
        frame=["xaf+lL(r)"],
        # scale=1,  # Adjust the scale
        # B="x10"
    )
    fig1.savefig(os.path.join(out_fig_dir,"01_distribution_Lr_{}_km.png".format(radius)),crop=True, dpi=300, transparent=False)
    
print("Done!")

surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


1.0


surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


2.0


surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


...

50.0


grdimage [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (14314)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (14314)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
grdimage [ERROR]: Unable to save current CPT file to /home/longhv/.gmt/sessions/gmt_session.253709/gmt.657.cpt !
grdimage [ERROR]: Failed to read CPT custom.cpt.
colorbar [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (14318)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (14318)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: 83 unusable

51.0


surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


52.0
Done!


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
# ======================================================================
'''
Plot the figure for each radius distances
'''
# ======================================================================
pwd = os.getcwd();
#
indata = pd.read_csv(os.path.join(pwd,"output","02_species_GR.txt"),sep="\s+",header=0)
# Output figure
out_fig_dir = os.path.join(pwd,"figures","GR_radius_paper")
os.makedirs(out_fig_dir,exist_ok=True)
# ======================================================================
for i,radius in enumerate(indata['radius_m']):
    print(radius)
    if i > 50:
        break
    if radius not in [5, 12]:
        continue
    # Extract the data
    data_now = indata[indata['radius_m']==radius]
    data_now.reset_index(drop=True,inplace=True)
    # Now plot the figure
    fig = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,np.ceil(int(indata['d2x'].max())/10)*10,0,np.ceil(int(indata['d2y'].max())/10)*10]
    # grid = pygmt.datasets.load_earth_relief(resolution="01s", region=area)
    # fig.grdimage(grid=grid, projection="X5i/5i", frame="a", cmap="geo")
    # fig.colorbar(frame=["a100", "x+lElevation", "y+lm"])
    fig.basemap(region=area, projection="X5i/5i", frame=['xafg+l"Easting(m)"','yafg+l"Northing(m)"', 'WSne+t"G(r) map with radius of {} (m)"'.format(radius)])
    gr_grd = pygmt.surface(x=data_now['d2x'], y=data_now['d2y'], z=data_now['gr'],
                            region=area, spacing='0.5/0.5') # 1 metter per 1 metter
    # pygmt.makecpt(cmap=["0   blue","1   white","{}   red".format(np.ceil((data_now['gr'].max())*10)/10)],series=[0, (np.ceil((data_now['gr'].max())*10)/10), 0.01])
    with open("custom.cpt", "w") as f:
        f.write(
            """
            0	blue	1	white
            1	white	{}	red
            B	blue
            F	white
            N	gray
            """.format(int(data_now['gr'].max()))
            )
    # -------------------------------------------------------------------
    # pygmt.makecpt(cmap=cmap,series=[0, 7, 0.01])
    fig.grdimage(grid=gr_grd,
                 # shading=True,
                 # projection="X5i/5i",
                 cmap="custom.cpt")
    fig.plot(x=data_now['d2x'],y=data_now['d2y'],
             style='c0.1c',
             pen='1p,black',
    )
    fig.plot(x=data_now[data_now['Point']=="S0058"]['d2x'],y=data_now[data_now['Point']=="S0058"]['d2y'],
             style='a0.5c',
             pen='1p,black',
             fill='yellow',
    )
    
    fig.colorbar(
        cmap="custom.cpt",
        position="JMR+o0.25i/0.0c+w5.0i/0.25c+n",
        box=None,
        frame=["xaf+lG(r)"],
        # scale=1,  # Adjust the scale
        # B="x10"
    )
    fig.savefig(os.path.join(out_fig_dir,"00_distribution_gr_{}_km.png".format(radius)),crop=True, dpi=300, transparent=False)
    # fig.show()
    # ---------------------------------------------------------------------------------------------------------------------------
    fig1 = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,np.ceil(int(indata['d2x'].max())/10)*10,0,np.ceil(int(indata['d2y'].max())/10)*10]
    # grid = pygmt.datasets.load_earth_relief(resolution="01s", region=area)
    # fig.grdimage(grid=grid, projection="X5i/5i", frame="a", cmap="geo")
    # fig.colorbar(frame=["a100", "x+lElevation", "y+lm"])
    fig1.basemap(region=area, projection="X5i/5i", frame=['xafg+l"Easting(m)"','yafg+l"Northing(m)"', 'WSne+t"L(r) map with radius of {} (m)"'.format(radius)])
    gr_grd = pygmt.surface(x=data_now['d2x'], y=data_now['d2y'], z=data_now['Lr'],
                            region=area, spacing='0.5/0.5') # 1 metter per 1 metter
    # pygmt.makecpt(cmap=["0   blue","1   white","{}   red".format(np.ceil((data_now['gr'].max())*10)/10)],series=[0, (np.ceil((data_now['gr'].max())*10)/10), 0.01])
    with open("custom_Kr.cpt", "w") as f:
        f.write(
            """
            {}	blue	0	white
            0	white	{}	red
            B	blue
            F	white
            N	gray
            """.format((data_now['Lr'].min()),(data_now['Lr'].max()))
            )
    # -------------------------------------------------------------------
    # pygmt.makecpt(cmap=cmap,series=[0, 7, 0.01])
    fig1.grdimage(grid=gr_grd,
                 # shading=True,
                 # projection="X5i/5i",
                 cmap="custom_Kr.cpt")
    
    fig1.plot(x=data_now['d2x'],y=data_now['d2y'],
             style='c0.1c',
             pen='1p,black',
    )
    fig1.plot(x=data_now[data_now['Point']=="S0058"]['d2x'],y=data_now[data_now['Point']=="S0058"]['d2y'],
             style='a0.5c',
             pen='1p,black',
             fill='yellow',
    )
    fig1.colorbar(
        cmap="custom_Kr.cpt",
        position="JMR+o0.25i/0.0c+w5.0i/0.25c+n",
        box=None,
        frame=["xaf+lL(r)"],
        # scale=1,  # Adjust the scale
        # B="x10"
    )
    fig1.savefig(os.path.join(out_fig_dir,"01_distribution_Lr_{}_km.png".format(radius)),crop=True, dpi=300, transparent=False)
    
print("Done!")

surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


1.0
2.0
3.0
4.0
5.0


surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


6.0
7.0
8.0
9.0
10.0
11.0
12.0


surface [WARNING]: 83 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
Done!


In [9]:
data_select = data_now[(data_now['d2x'] > 28) & (data_now['d2x'] < 32) & (data_now['d2y'] > 53) & (data_now['d2x'] < 55)]
data_select

,Point,Longitude,Latitude,radius_m,gr,Lr,distance,d2x,d2y
98,S0216,108.575694,14.500013,12.0,0.645612,0.447768,105.880827,31.648999,101.040044
99,S0217,108.575691,14.499991,12.0,0.968418,-0.523725,103.402131,31.336393,98.539491
101,S0219,108.575689,14.499939,12.0,1.936837,1.917028,97.886555,31.185485,92.786007
102,S0220,108.575689,14.499939,12.0,1.936837,1.917028,97.886555,31.185485,92.786007
103,S0221,108.575675,14.499838,12.0,1.936837,-1.219916,86.817057,29.611672,81.610969
233,S0373,108.575683,14.499754,12.0,1.614031,0.755167,78.482030,30.517173,72.305817
279,S0709,108.575673,14.499597,12.0,1.614031,-1.219916,62.350287,29.450009,54.956849
369,S0752,108.575689,14.500085,12.0,0.807015,-1.585450,113.341740,31.163905,108.973212
402,S0269,108.575683,14.500095,12.0,0.968418,-1.585450,114.207247,30.506347,110.057522
405,S0272,108.575667,14.500037,12.0,0.484209,0.447768,107.563271,28.824732,103.629108


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
# ======================================================================
'''
Plot the figure for each radius distances
'''
# ======================================================================
pwd = os.getcwd();
#
indata = pd.read_csv(os.path.join(pwd,"output","02_species_GR.txt"),sep="\s+",header=0)
# Output figure
out_fig_dir = os.path.join(pwd,"figures","GR_Lr_tree")
os.makedirs(out_fig_dir,exist_ok=True)
# ======================================================================
# Extract each point data
for i,point in enumerate(indata['Point'].unique()):
    print(point)
    if i > len(indata['Point'].unique()):
        break
    # if i != 2: 
    #     continue
    datanow = indata[indata['Point']==point];
    datanow.reset_index(drop=True,inplace=True)
    # ------------------------------------------------------------------
    fig = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,int(datanow['radius_m'].max()),0,datanow['gr'].max()+1]
    fig.basemap(region=area, projection="X5i/2.5i", frame=['xafg+l"Radius(m)"','yafg+l"G(r)"', 'WSne+t"G(r) for tree {}"'.format(point)])
    fig.plot(x=datanow['radius_m'],y=datanow['gr'],pen="1.5p,black")
    fig.plot(x=datanow['radius_m'],y=datanow['gr'],style="c0.25c",pen="0.5p,black",fill="red")
    fig.plot(x=[0,1000],y=[1,1],pen="1.0p,purple,-")
    #
    fig.shift_origin(xshift="+w1c")
    # area1 = [0,int(datanow['radius_m'].max()),datanow['Lr'].min()-1,datanow['Lr'].max()+1]
    area1 = [0,int(datanow['radius_m'].max()),-10,datanow['Lr'].max()+1]
    fig.basemap(region=area1, projection="X5i/2.5i", frame=['xafg+l"Radius(m)"','yafg+l"L(r)"', 'wSnE+t"L(r) for tree {}"'.format(point)])
    fig.plot(x=datanow['radius_m'],y=datanow['Lr'],pen="1.5p,black")
    fig.plot(x=datanow['radius_m'],y=datanow['Lr'],style="c0.25c",pen="0.5p,black",fill="blue")
    fig.plot(x=[0,1000],y=[0,0],pen="1.0p,purple,-")
    fig.savefig(os.path.join(out_fig_dir,"00_{}.png".format(point)),crop=True, dpi=300, transparent=False)
    # fig.show()
print("Done!")

S0113
S0114
S0115
S0117
S0118
S0119
S0120
S0121
S0122
S0123
S0124
S0127
S0128
S0129
S0130
S0131
S0132
S0133
S0134
S0136
S0137
S0138
S0139
S0141
S0142
S0143
S0144
S0145
S0146
S0147
S0148
S0149
S0150
S0151
S0152
S0153
S0154
S0155
S0156
S0157
S0158
S0159
S0160
S0161
S0162
S0163
S0164
S0165
S0166
S0167
S0168
S0169
S0170
S0171
S0172
S0173
S0174
S0175
S0176
S0177
S0178
S0179
S0180
S0181
S0182
S0183
S0184
S0185
S0186
S0187
S0188
S0189
S0190
S0191
S0192
S0193
S0194
S0195
S0196
S0197
S0198
S0199
S0200
S0201
S0202
S0203
S0204
S0205
S0206
S0207
S0208
S0209
S0210
S0211
S0212
S0213
S0214
S0215
S0216
S0217
S0218
S0219
S0220
S0221
S0222
S0223
S0224
S0225
S0226
S0227
S0228
S0229
S0230
S0231
S0232
S0233
S0234
S0235
S0236
S0237
S0238
S0239
S0240
S0241
S0417
S0418
S0419
S0420
S0421
S0422
S0423
S0424
S0425
S0426
S0427
S0428
S0429
S0430
S0431
S0432
S0433
S0434
S0435
S0436
S0437
S0438
S0439
S0440
S0441
S0442
S0443
S0444
S0445
S0446
S0447
S0448
S0449
S0450
S0451
S0452
S0453
S0454
S0621
S0622
S0623
S0624
S062